In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import BernoulliRBM
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

In [3]:
df = pd.read_csv(r"C:\Users\rosem\Downloads\Paper_works\CSVs\Feature_engg_Paper2.csv")
df = df[df['Region'].isin(['CAL','TEX'])]
df

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Rolling_Mean_24H,Rolling_Mean_3H_Emission,Rolling_Mean_24H_Emission,Total_Gen,Renewable_Pct,Fossil_Pct,Renewable_Demand,Fossil_Demand,Demand_diff1,Demand_diff24
0,2018-07-01 17:00:00,12.5,28976.042,21138.304,-6528.677,28386.220,8.764,7725.951,2261.073,3089.741,...,42604.509,27096.582,17297.042,17521.203,42.950,44.145,1244530.613,1279144.078,89.458,172.875
1,2018-07-01 18:00:00,12.5,29065.500,21217.885,-6554.521,28486.010,8.792,7761.917,2261.062,3110.594,...,41847.195,8244.812,17052.778,17577.073,42.927,44.209,1247693.394,1284966.639,89.458,172.875
2,2018-07-01 19:00:00,12.5,29154.958,21297.467,-6580.365,28585.800,8.819,7797.882,2261.052,3131.446,...,45629.697,24266.186,19327.269,17632.943,42.904,44.273,1250856.313,1290788.823,89.458,172.875
3,2018-07-01 20:00:00,12.5,29244.417,21377.049,-6606.208,28685.590,8.847,7833.847,2261.042,3152.299,...,48609.446,6436.826,20904.933,17688.812,42.881,44.337,1254019.366,1296610.635,89.458,172.875
4,2018-07-01 21:00:00,12.5,29333.875,21456.630,-6632.052,28785.380,8.875,7869.812,2261.031,3173.151,...,49142.567,6326.084,21205.564,17744.682,42.858,44.400,1257182.553,1302432.078,89.458,172.875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525055,2023-06-29 20:00:00,12.5,64967.778,65025.979,58.042,65732.083,8989.486,27552.910,4647.181,32.896,...,49409.849,30361.664,19267.173,46077.069,10.607,79.307,689127.079,5152407.218,-65.642,-1360.549
525056,2023-06-29 21:00:00,12.5,64902.135,64963.474,61.177,65699.302,8993.729,27593.401,4647.292,33.224,...,46137.266,13730.128,18243.586,46106.208,10.566,79.354,685784.748,5150245.412,-65.642,-1414.266
525057,2023-06-29 22:00:00,12.5,64836.493,64900.969,64.312,65666.521,8997.972,27633.892,4647.403,33.552,...,45454.091,12961.713,16905.630,46135.347,10.526,79.401,682451.108,5148073.607,-65.642,-1467.983
525058,2023-06-29 23:00:00,12.5,64770.851,64838.464,67.448,65633.740,9002.215,27674.384,4647.514,33.880,...,39779.804,15939.369,14414.641,46164.486,10.485,79.448,679126.143,5145891.821,-65.642,-1521.700


In [7]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

# --- Define Attention Layer ---
import tensorflow as tf
from tensorflow.keras.layers import Layer

class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)
    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")
        super(Attention, self).build(input_shape)
    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

# --- Utility metrics ---
def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + 1e-8))) * 100

# --- Your setup ---
region_code = 'CAL'  # or 'TEX'
feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df.set_index('Date', inplace=True)

features = region_df[feature_cols].values
targets = region_df[target_cols].values

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

lookback = 30
X_seq, y_seq = create_sequences(X_scaled, y_scaled, lookback=lookback)

split_ratios = [0.8, 0.75, 0.7, 0.65, 0.6]
split_names = ["80-20", "75-25", "70-30", "65-35", "60-40"]
results = []

for split, split_name in zip(split_ratios, split_names):
    n_train = int(split * len(X_seq))
    X_train, X_test = X_seq[:n_train], X_seq[n_train:]
    y_train, y_test = y_seq[:n_train], y_seq[n_train:]

    n_features = X_train.shape[2]
    model = Sequential([
        Input(shape=(lookback, n_features)),
        LSTM(64, return_sequences=True),
        Attention(),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1])  # 2 for [Demand, CO2]
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0, validation_data=(X_test, y_test))

    # Predict and inverse scale
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    results.append({
        'Split': split_name,
        'Region': 'California' if region_code == 'CAL' else 'Texas',
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand(%)': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2(%)': mape(y_true[:, 1], y_pred[:, 1])
    })

results_df = pd.DataFrame(results)
print(results_df)

274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
342/342 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
479/479 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
547/547 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
   Split      Region  MAE_Demand  RMSE_Demand  MAPE_Demand(%)     MAE_CO2  \
0  80-20  California  215.236971   300.589303        0.692811  163.214581   
1  75-25  California  232.490806   301.622266        0.792500  234.426068   
2  70-30  California  279.271436   337.390945        0.929699  374.041414   
3  65-35  California  380.055823   433.637657        1.277430  411.314110   
4  60-40  California  303.953559   390.077497        0.959953  398.962510   

     RMSE_CO2  MAPE_CO2(%)  
0  200.505207     3.276000  
1  278.947840     4.789899  
2  415.438623     8.387262  
3  457.320512     9.032513  
4  474.324407     8.332728  


In [9]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

# --- Define Attention Layer ---
import tensorflow as tf
from tensorflow.keras.layers import Layer

class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)
    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")
        super(Attention, self).build(input_shape)
    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

# --- Utility metrics ---
def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + 1e-8))) * 100

# --- Your setup ---
region_code = 'TEX'  # or 'TEX'
feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df.set_index('Date', inplace=True)

features = region_df[feature_cols].values
targets = region_df[target_cols].values

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

lookback = 30
X_seq, y_seq = create_sequences(X_scaled, y_scaled, lookback=lookback)

split_ratios = [0.8, 0.75, 0.7, 0.65, 0.6]
split_names = ["80-20", "75-25", "70-30", "65-35", "60-40"]
results = []

for split, split_name in zip(split_ratios, split_names):
    n_train = int(split * len(X_seq))
    X_train, X_test = X_seq[:n_train], X_seq[n_train:]
    y_train, y_test = y_seq[:n_train], y_seq[n_train:]

    n_features = X_train.shape[2]
    model = Sequential([
        Input(shape=(lookback, n_features)),
        LSTM(64, return_sequences=True),
        Attention(),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1])  # 2 for [Demand, CO2]
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0, validation_data=(X_test, y_test))

    # Predict and inverse scale
    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    results.append({
        'Split': split_name,
        'Region': 'California' if region_code == 'CAL' else 'Texas',
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand(%)': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2(%)': mape(y_true[:, 1], y_pred[:, 1])
    })

results_df = pd.DataFrame(results)
print(results_df)

274/274 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
342/342 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
411/411 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step
547/547 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
   Split Region  MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0  80-20  Texas  237.372008   310.087758        0.499923  1120.718486   
1  75-25  Texas  885.031484  1527.722525        1.578721   900.225723   
2  70-30  Texas  546.913111   717.745050        1.090010  1384.863672   
3  65-35  Texas  411.171292   542.806025        0.865740   751.932705   
4  60-40  Texas  724.108845  1118.532612        1.370614   860.672512   

      RMSE_CO2  MAPE_CO2(%)  
0  1376.910329     8.209242  
1  1149.902774     6.432298  
2  1665.165041     9.011465  
3   971.102026     5.390543  
4  1053.231111     5.787367  
